In [86]:
import pandas as pd
import warnings
import os

import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.preprocessing import sequence
from sklearn.model_selection import KFold
# fix random seed for reproducibility
np.random.seed(7)

warnings.filterwarnings('ignore')

In [87]:
df_all = pd.read_pickle('/Users/nicholasutikal/Documents/private code/DS/bookie/udacity_bookie/udacity_ML/football_results/ms/pickle_files/df_rnn_base.pkl')

In [88]:
df_all=df_all[df_all['status']=='complete']

In [89]:
teams_dict = {}
teams_dict_full = {}
l_teams = df_all['home_team_name'].unique()

for i in l_teams:
    df_home=df_all.loc[df_all.home_team_name ==i]
    df_away=df_all.loc[df_all.away_team_name ==i]
    df_team=pd.concat([df_home, df_away])
    df_team.sort_values('timestamp', inplace=True)
    df_team.dropna(axis=0,inplace=True)
    teams_dict[i] = df_team.index
    teams_dict_full[i] = df_team

In [90]:
####################################
df_all=teams_dict_full['Stuttgart']
####################################

In [91]:
df_all

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,H_result_1,H_result_2,A_result_1,A_result_2,H_HTGDIFF_1,H_HTGDIFF_2,A_ATGDIFF_1,A_ATGDIFF_2,H_AVGHTGDIFF_1,A_AVGATGDIFF_1
342,1505838600,Sep 19 2017 - 4:30pm,complete,44063.0,Borussia M'gladbach,Stuttgart,0,0.0,1.50,0.00,...,1.0,3.0,3.0,3.0,-1.0,1.0,-2.0,-2.0,0.00,0.00
356,1506173400,Sep 23 2017 - 1:30pm,complete,55202.0,Stuttgart,Augsburg,0,0.0,3.00,1.50,...,3.0,3.0,1.0,3.0,1.0,1.0,1.0,-1.0,0.00,0.00
364,1506778200,Sep 30 2017 - 1:30pm,complete,51500.0,Eintracht Frankfurt,Stuttgart,0,0.0,0.00,0.00,...,1.0,1.0,3.0,3.0,-1.0,-1.0,-2.0,-2.0,-1.00,0.00
369,1507919400,Oct 13 2017 - 6:30pm,complete,58716.0,Stuttgart,Köln,0,0.0,2.33,0.25,...,2.0,3.0,2.0,3.0,0.0,1.0,0.0,-5.0,0.00,-2.25
379,1508592600,Oct 21 2017 - 1:30pm,complete,42558.0,RB Leipzig,Stuttgart,0,0.0,2.33,0.00,...,3.0,2.0,3.0,3.0,1.0,0.0,-1.0,-2.0,0.75,-1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1260,1603477800,Oct 23 2020 - 6:30pm,complete,0.0,Stuttgart,Köln,Guido Winkmann,5.0,0.50,0.00,...,2.0,1.0,3.0,3.0,0.0,-1.0,-1.0,-5.0,0.75,-2.00
1269,1604086200,Oct 30 2020 - 7:30pm,complete,0.0,Schalke 04,Stuttgart,Benjamin Brand,6.0,0.50,3.00,...,2.0,1.0,1.0,1.0,0.0,-2.0,2.0,3.0,-1.25,0.75
1283,1604759400,Nov 07 2020 - 2:30pm,complete,0.0,Stuttgart,Eintracht Frankfurt,Sascha Stegemann,7.0,0.67,1.33,...,2.0,2.0,3.0,2.0,0.0,0.0,-5.0,0.0,0.50,-0.75
1289,1605969000,Nov 21 2020 - 2:30pm,complete,0.0,Hoffenheim,Stuttgart,Daniel Siebert,8.0,1.00,2.33,...,1.0,1.0,2.0,1.0,-2.0,-1.0,0.0,2.0,1.00,1.25


In [92]:
df_matches_with_aa_numeric = df_all._get_numeric_data()
df_matches_with_aa_numeric.drop(
    ['timestamp', 'goal_diff', 'result', 'home_team_goal_count', 'away_team_goal_count', 'Game Week', 'attendance'], axis=1, inplace=True)

In [93]:
df_matches_with_aa_numeric

,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,...,H_result_1,H_result_2,A_result_1,A_result_2,H_HTGDIFF_1,H_HTGDIFF_2,A_ATGDIFF_1,A_ATGDIFF_2,H_AVGHTGDIFF_1,A_AVGATGDIFF_1
342,1.50,0.00,1.82,1.00,2,0,0,0,4,1,...,1.0,3.0,3.0,3.0,-1.0,1.0,-2.0,-2.0,0.00,0.00
356,3.00,1.50,2.00,1.12,0,0,0,0,5,3,...,3.0,3.0,1.0,3.0,1.0,1.0,1.0,-1.0,0.00,0.00
364,0.00,0.00,1.65,1.00,3,1,1,0,3,6,...,1.0,1.0,3.0,3.0,-1.0,-1.0,-2.0,-2.0,-1.00,0.00
369,2.33,0.25,2.00,0.47,3,1,1,0,5,4,...,2.0,3.0,2.0,3.0,0.0,1.0,0.0,-5.0,0.00,-2.25
379,2.33,0.00,1.82,1.00,1,1,1,0,3,5,...,3.0,2.0,3.0,3.0,1.0,0.0,-1.0,-2.0,0.75,-1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1260,0.50,0.00,0.60,1.25,2,2,1,1,3,5,...,2.0,1.0,3.0,3.0,0.0,-1.0,-1.0,-5.0,0.75,-2.00
1269,0.50,3.00,0.50,2.00,2,1,1,0,3,4,...,2.0,1.0,1.0,1.0,0.0,-2.0,2.0,3.0,-1.25,0.75
1283,0.67,1.33,0.60,1.20,4,2,2,0,10,8,...,2.0,2.0,3.0,2.0,0.0,0.0,-5.0,0.0,0.50,-0.75
1289,1.00,2.33,1.00,2.00,6,3,1,2,3,4,...,1.0,1.0,2.0,1.0,-2.0,-1.0,0.0,2.0,1.00,1.25


In [94]:
l_droped_rows=df_matches_with_aa_numeric[df_matches_with_aa_numeric.isnull().any(axis=1)].index.values

In [95]:
l_droped_rows

array([], dtype=int64)

In [96]:
df_matches_with_aa_numeric.dropna(axis=0,inplace=True)
df_matches_with_aa_numeric.isnull().sum(axis=0)

Pre-Match PPG (Home)    0
Pre-Match PPG (Away)    0
home_ppg                0
away_ppg                0
total_goal_count        0
                       ..
H_HTGDIFF_2             0
A_ATGDIFF_1             0
A_ATGDIFF_2             0
H_AVGHTGDIFF_1          0
A_AVGATGDIFF_1          0
Length: 75, dtype: int64

In [97]:
df_matches_with_aa_numeric

,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,...,H_result_1,H_result_2,A_result_1,A_result_2,H_HTGDIFF_1,H_HTGDIFF_2,A_ATGDIFF_1,A_ATGDIFF_2,H_AVGHTGDIFF_1,A_AVGATGDIFF_1
342,1.50,0.00,1.82,1.00,2,0,0,0,4,1,...,1.0,3.0,3.0,3.0,-1.0,1.0,-2.0,-2.0,0.00,0.00
356,3.00,1.50,2.00,1.12,0,0,0,0,5,3,...,3.0,3.0,1.0,3.0,1.0,1.0,1.0,-1.0,0.00,0.00
364,0.00,0.00,1.65,1.00,3,1,1,0,3,6,...,1.0,1.0,3.0,3.0,-1.0,-1.0,-2.0,-2.0,-1.00,0.00
369,2.33,0.25,2.00,0.47,3,1,1,0,5,4,...,2.0,3.0,2.0,3.0,0.0,1.0,0.0,-5.0,0.00,-2.25
379,2.33,0.00,1.82,1.00,1,1,1,0,3,5,...,3.0,2.0,3.0,3.0,1.0,0.0,-1.0,-2.0,0.75,-1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1260,0.50,0.00,0.60,1.25,2,2,1,1,3,5,...,2.0,1.0,3.0,3.0,0.0,-1.0,-1.0,-5.0,0.75,-2.00
1269,0.50,3.00,0.50,2.00,2,1,1,0,3,4,...,2.0,1.0,1.0,1.0,0.0,-2.0,2.0,3.0,-1.25,0.75
1283,0.67,1.33,0.60,1.20,4,2,2,0,10,8,...,2.0,2.0,3.0,2.0,0.0,0.0,-5.0,0.0,0.50,-0.75
1289,1.00,2.33,1.00,2.00,6,3,1,2,3,4,...,1.0,1.0,2.0,1.0,-2.0,-1.0,0.0,2.0,1.00,1.25


In [98]:
df_norm = (df_matches_with_aa_numeric - df_matches_with_aa_numeric.min()) / (
        df_matches_with_aa_numeric.max() - df_matches_with_aa_numeric.min())

In [99]:
df_all_clean=df_all.drop(list(l_droped_rows))

In [100]:
df_norm.values

array([[0.5       , 0.        , 0.67094017, ..., 0.4       , 0.24      ,
        0.65      ],
       [1.        , 0.5       , 0.74786325, ..., 0.5       , 0.24      ,
        0.65      ],
       [0.        , 0.        , 0.5982906 , ..., 0.4       , 0.08      ,
        0.65      ],
       ...,
       [0.22333333, 0.44333333, 0.14957265, ..., 0.6       , 0.32      ,
        0.5       ],
       [0.33333333, 0.77666667, 0.32051282, ..., 0.8       , 0.4       ,
        0.9       ],
       [0.25      , 0.75      , 0.14957265, ..., 0.7       , 0.2       ,
        0.75      ]])

In [121]:
from sklearn.model_selection import train_test_split
def prepare_datasets(test_size, validation_size):
    X=df_norm.values
    y=df_all_clean['result'].values
    


    X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=test_size, random_state=42)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)
    
    return X_train, X_validation, X_test, y_train, y_validation, y_test

In [102]:
def build_model(input_shape):
    """Generates RNN-LSTM model
    :param input_shape (tuple): Shape of input set
    :return model: RNN-LSTM model
    """

    # build network topology
    model = Sequential()

    # 2 LSTM layers
    model.add(LSTM(64, activation="relu",input_shape=input_shape, return_sequences=True))
    model.add(LSTM(64))

    # dense layer
    model.add(Dense(64, activation='relu'))
#     model.add(keras.layers.Dropout(0.3))

    # output layer
    model.add(Dense(3, activation='softmax'))

    return model

In [140]:
X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2)
# X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_train = np.reshape(100,(X_train.shape[0], X_train.shape[1]))
# create network
# X_train = X_train[..., None]


ValueError: cannot reshape array of size 1 into shape (43,75)

In [104]:
X_test.shape

(18, 75)

In [105]:
input_shape = (X_train.shape[0], X_train.shape[1]) 
print("x-train-shape= ",X_train.shape)
print("y-train-shape= ",y_train.shape)

x-train-shape=  (43, 75)
y-train-shape=  (43,)


In [106]:
input_shape

(43, 75)

In [107]:
model = build_model(input_shape)

In [108]:
# from tensorflow.keras.layers import *
# from tensorflow.keras.models import *
# import numpy as np
# # train_data = np.random.randint(0,3, (100, 4000))
# # y_labels = np.random.randint(0,2, (100,))

# input_ = Input(shape=(75)) # shape from train_data
# embedded = Embedding(36, 256, input_length = 75)(input_) 
# lstm = LSTM(256, return_sequences=True)(embedded) # --> ERROR
# dense = Dense(3, activation='softmax')(lstm) 

# model = Model(input_, dense)
# model.summary()

In [109]:
# optimiser = Adam(learning_rate=0.001)
# model.compile(optimizer=optimiser,
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

In [84]:
# history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=5, epochs=20)

In [138]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
input_array = np.random.randint(100, size=(250, 2000))
input_y = np.random.randint(7, size = (250))

Y_dumy = pd.get_dummies(input_y)
X_train, X_test, Y_train, Y_test = train_test_split(input_array, Y_dumy,  
test_size=0.1, random_state=0)

model = Sequential()
model.add(Embedding(input_dim = 100, output_dim = 64, input_length=100))
model.add(LSTM(100, return_sequences=True))
model.output_shape

(None, 100, 100)

In [132]:
df_norm.shape

(72, 75)

In [123]:
#######

In [129]:
input_array.shape

(250, 2000)

In [118]:
Y_dumy.shape

(250, 7)

In [119]:
X_train.shape

(225, 2000)

In [120]:
y_train.shape

(43,)

In [139]:
#Output shape should be:
#model.output_shape = (None, 2000, 64)
#3D tensor with shape: (batch_size, sequence_length, output_dim)

model.add(LSTM(200, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics= 
['accuracy'])
model.fit(X_train, Y_train, epochs=2, verbose=True, validation_data= 
(X_test, Y_test))

Epoch 1/2
8/8 [==============================] - 36s 4s/step - loss: 1.9474 - accuracy: 0.1467 - val_loss: 1.9349 - val_accuracy: 0.1600
Epoch 2/2
8/8 [==============================] - 34s 4s/step - loss: 1.9372 - accuracy: 0.2667 - val_loss: 1.9244 - val_accuracy: 0.2000


In [ ]:
Y_dumy.shape

In [ ]:
X_train.shape

In [110]:
X_train.shape

(43, 75)

In [112]:
y_train.shape

(43,)